In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn import tree, ensemble, metrics, cross_validation, externals
%matplotlib inline


pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

plt.style.use('ggplot')

In [3]:
userhome = os.path.expanduser('~')
df = pd.read_csv(os.path.join(userhome,'Desktop','DS-SF-23','lending-club-loan-data','loan.csv'), index_col = 'id', low_memory=False)
%load_ext gvmagic


In [4]:
df1 = df

In [3]:
df1.count()

NameError: name 'df1' is not defined

In [219]:
# df[['annual_inc','loan_amnt']].plot(kind='box') this will be used to see the box plot for these two

In [220]:
# mpl.rcParams['agg.path.chunksize'] = 10000

In [6]:
df.describe()

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,...,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,8.873790e+05,887379.000000,887379.000000,887379.000000,887379.000000,...,21372.000000,8.171030e+05,21372.000000,21372.000000,21372.000000
mean,3.500182e+07,14755.264605,14741.877625,14702.464383,13.246740,...,60.831939,3.206862e+04,0.943945,1.537058,1.977307
std,2.411335e+07,8435.455601,8429.897657,8442.106732,4.381867,...,20.013254,3.749826e+04,1.446872,2.723339,2.874067
min,7.047300e+04,500.000000,500.000000,0.000000,5.320000,...,0.000000,0.000000e+00,0.000000,0.000000,-4.000000
25%,1.087713e+07,8000.000000,8000.000000,8000.000000,9.990000,...,47.700000,1.390000e+04,0.000000,0.000000,0.000000
50%,3.709528e+07,13000.000000,13000.000000,13000.000000,12.990000,...,61.900000,2.370000e+04,0.000000,0.000000,2.000000
75%,5.847135e+07,20000.000000,20000.000000,20000.000000,16.200000,...,75.200000,3.980000e+04,1.000000,2.000000,3.000000
max,7.354484e+07,35000.000000,35000.000000,35000.000000,28.990000,...,151.400000,9.999999e+06,17.000000,35.000000,32.000000


In [7]:
len(df.columns)

73

In [8]:
df.drop(['emp_title', 'url','title','policy_code','desc'], axis = 1, inplace = True)

In [10]:
len(df.columns)

68

In [9]:
#df.columns

In [11]:
df.loan_status.value_counts()

Current                                                601779
Fully Paid                                             207723
Charged Off                                             45248
Late (31-120 days)                                      11591
Issued                                                   8460
In Grace Period                                          6253
Late (16-30 days)                                        2357
Does not meet the credit policy. Status:Fully Paid       1988
Default                                                  1219
Does not meet the credit policy. Status:Charged Off       761
Name: loan_status, dtype: int64

In [12]:
df['FullyPaid'] = (df.loan_status == 'Fully Paid') * 1.0

In [13]:
df['FullyPaid'].value_counts()

0.0    679656
1.0    207723
Name: FullyPaid, dtype: int64

In [14]:
df.drop(['loan_status'], axis = 1, inplace = True)

In [15]:
len(df.columns)

68

In [16]:
df.verification_status.value_counts()

Source Verified    329558
Verified           291071
Not Verified       266750
Name: verification_status, dtype: int64

In [17]:
df['is_Verified'] = ((df.verification_status == 'Verified') | (df.verification_status == 'Source Verified')) * 1.0

In [18]:
df['is_Verified'].value_counts()

1.0    620629
0.0    266750
Name: is_Verified, dtype: int64

In [19]:
len(df.columns)

69

In [20]:
df.drop(['verification_status'], axis = 1, inplace = True)

In [21]:
len(df.columns)

68

In [22]:
# df.head(2)

In [23]:
df.term.unique()

array([' 36 months', ' 60 months'], dtype=object)

In [24]:
df['term'] = df.term.str.replace('months', '')

In [25]:
df.term.unique()

array([' 36 ', ' 60 '], dtype=object)

In [26]:
df.term = pd.to_numeric(df.term)

In [245]:
#df.head(2)

In [27]:
df.grade.unique()

array(['B', 'C', 'A', 'E', 'F', 'D', 'G'], dtype=object)

In [28]:
df['grade'] = df.grade.str.replace('A', '1')
df['grade'] = df.grade.str.replace('B', '2')
df['grade'] = df.grade.str.replace('C', '3')
df['grade'] = df.grade.str.replace('D', '4')
df['grade'] = df.grade.str.replace('E', '5')
df['grade'] = df.grade.str.replace('F', '6')
df['grade'] = df.grade.str.replace('G', '7')

In [250]:
#df.head(2)

In [29]:
df.grade = pd.to_numeric(df.grade)

In [30]:
df.emp_length.unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', 'n/a'], dtype=object)

In [31]:
df.emp_length.value_counts()

10+ years    291569
2 years       78870
< 1 year      70605
3 years       70026
1 year        57095
              ...  
n/a           44825
7 years       44594
8 years       43955
6 years       42950
9 years       34657
Name: emp_length, dtype: int64

In [32]:
df['emp_length'] = df.emp_length.str.replace('year', '')
df['emp_length'] = df.emp_length.str.replace('s', '')
df['emp_length'] = df.emp_length.str.replace('n/a', '0')
df['emp_length'] = df.emp_length.str.replace('< 1', '0.5')
df['emp_length'] = df.emp_length.str.replace('+', '')

In [33]:
df.emp_length.value_counts()

10      291569
2        78870
0.5      70605
3        70026
1        57095
         ...  
0        44825
7        44594
8        43955
6        42950
9        34657
Name: emp_length, dtype: int64

In [34]:
df.emp_length = pd.to_numeric(df.emp_length)

In [35]:
df.head(2)

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,...,inq_fi,total_cu_tl,inq_last_12m,FullyPaid,is_Verified
id,,,,,,,,,,,
1077501,1296599,5000.0,5000.0,4975.0,36,...,NaN,NaN,NaN,1.0,1.0
1077430,1314167,2500.0,2500.0,2500.0,60,...,NaN,NaN,NaN,0.0,1.0


In [36]:
df.home_ownership.value_counts()

MORTGAGE    443557
RENT        356117
OWN          87470
OTHER          182
NONE            50
ANY              3
Name: home_ownership, dtype: int64

In [37]:
df[df.home_ownership == 'ANY']['loan_amnt']

id
26170263     5000.0
66544417    34300.0
55948695    24000.0
Name: loan_amnt, dtype: float64

In [38]:
df[df.home_ownership == 'NONE']['loan_amnt'].head(2)

id
231321    10000.0
121568     4000.0
Name: loan_amnt, dtype: float64

In [39]:
df['is_FullyOwned'] = (df.home_ownership == 'OWN') * 1.0

In [40]:
df.is_FullyOwned.value_counts()

0.0    799909
1.0     87470
Name: is_FullyOwned, dtype: int64

In [41]:
len(df.columns)

69

In [42]:
df.drop(['home_ownership'], axis=1, inplace = True)

In [43]:
df.columns

Index([u'member_id', u'loan_amnt', u'funded_amnt', u'funded_amnt_inv', u'term',
       u'int_rate', u'installment', u'grade', u'sub_grade', u'emp_length',
       u'annual_inc', u'issue_d', u'pymnt_plan', u'purpose', u'zip_code',
       u'addr_state', u'dti', u'delinq_2yrs', u'earliest_cr_line',
       u'inq_last_6mths', u'mths_since_last_delinq', u'mths_since_last_record',
       u'open_acc', u'pub_rec', u'revol_bal', u'revol_util', u'total_acc',
       u'initial_list_status', u'out_prncp', u'out_prncp_inv', u'total_pymnt',
       u'total_pymnt_inv', u'total_rec_prncp', u'total_rec_int',
       u'total_rec_late_fee', u'recoveries', u'collection_recovery_fee',
       u'last_pymnt_d', u'last_pymnt_amnt', u'next_pymnt_d',
       u'last_credit_pull_d', u'collections_12_mths_ex_med',
       u'mths_since_last_major_derog', u'application_type',
       u'annual_inc_joint', u'dti_joint', u'verification_status_joint',
       u'acc_now_delinq', u'tot_coll_amt', u'tot_cur_bal', u'open_acc_6m',
   

In [44]:
df.verification_status_joint.unique()

array([nan, 'Not Verified', 'Verified', 'Source Verified'], dtype=object)

In [45]:
df.verification_status_joint.value_counts()

Not Verified       283
Verified           167
Source Verified     61
Name: verification_status_joint, dtype: int64

In [46]:
df['is_JointVerified'] = ((df.verification_status_joint == 'Verified') | (df.verification_status_joint == 'Source Verified')) * 1.0

In [47]:
df.is_JointVerified.value_counts()

0.0    887151
1.0       228
Name: is_JointVerified, dtype: int64

In [51]:
df.drop(['verification_status_joint'], axis = 1, inplace = True)

ValueError: labels ['verification_status_joint'] not contained in axis

In [54]:
len(df.columns)

68

In [55]:
df.application_type.value_counts()

INDIVIDUAL    886868
JOINT            511
Name: application_type, dtype: int64

In [56]:
df['is_Individual'] = (df.application_type == 'INDIVIDUAL')* 1.0

In [57]:
df.is_Individual.value_counts()

1.0    886868
0.0       511
Name: is_Individual, dtype: int64

In [58]:
df.drop(['application_type'], axis = 1, inplace = True)

In [59]:
len(df.columns)

68

In [60]:
df.drop(['issue_d','last_pymnt_d','last_credit_pull_d','next_pymnt_d'], axis=1, inplace=True)

In [61]:
len(df.columns)

64

In [64]:
df[range(0,10)].head(50)

,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length
id,,,,,,,,,,
1077501,1296599,5000.0,5000.0,4975.0,36,10.65,162.87,2,B2,10.0
1077430,1314167,2500.0,2500.0,2500.0,60,15.27,59.83,3,C4,0.5
1077175,1313524,2400.0,2400.0,2400.0,36,15.96,84.33,3,C5,10.0
1076863,1277178,10000.0,10000.0,10000.0,36,13.49,339.31,3,C1,10.0
1075358,1311748,3000.0,3000.0,3000.0,60,12.69,67.79,2,B5,1.0
...,...,...,...,...,...,...,...,...,...,...
1051117,1282787,14000.0,14000.0,14000.0,36,9.91,451.15,2,B1,6.0
1069465,1304521,5000.0,5000.0,5000.0,36,8.90,158.77,1,A5,10.0
1069283,1304166,10000.0,10000.0,10000.0,36,10.65,325.74,2,B2,6.0


In [65]:
df.pymnt_plan.value_counts()

n    887369
y        10
Name: pymnt_plan, dtype: int64

In [66]:
df['is_PaymentPlan'] = (df.pymnt_plan == 'y') * 1.0

In [67]:
df.is_PaymentPlan.value_counts()

0.0    887369
1.0        10
Name: is_PaymentPlan, dtype: int64

In [316]:
#df.earliest_cr_line = pd.to_datetime(df.earliest_cr_line) 

In [68]:
df.drop(['earliest_cr_line'], axis = 1, inplace = True)

In [72]:
df.drop(['pymnt_plan'], axis = 1, inplace = True)

In [73]:
len(df.columns)

63

In [74]:
df.initial_list_status.value_counts()

f    456848
w    430531
Name: initial_list_status, dtype: int64

In [75]:
df['is_Whole'] = (df.initial_list_status == 'w')*1.0

In [76]:
df.is_Whole.value_counts()

0.0    456848
1.0    430531
Name: is_Whole, dtype: int64

In [77]:
len(df.columns)

64

In [79]:
df.drop(['purpose','zip_code','addr_state','initial_list_status'], axis=1, inplace=True)

In [80]:
len(df.columns)

60

In [81]:
df.drop(['sub_grade'], axis=1,inplace=True)

In [82]:
df = df.fillna(0)

In [83]:
df.drop(['member_id'], axis=1, inplace=True)

In [141]:
#df

In [111]:
df.FullyPaid.value_counts()

0.0    679656
1.0    207723
Name: FullyPaid, dtype: int64

In [ ]:
#randomly taking the same amoung of data, and creating a new data set

In [133]:
y_0 = df[df.FullyPaid == 0.0]
y_1 = df[df.FullyPaid == 1.0]
n = min([len(y_0), len(y_1)])
y_0 = y_0.sample(n = n, random_state = 0)
y_1 = y_1.sample(n = n, random_state = 0)
df_strat = pd.concat([y_0, y_1])

In [154]:
df_strat.FullyPaid.value_counts()

1.0    207723
0.0    207723
Name: FullyPaid, dtype: int64

Index([u'loan_amnt', u'funded_amnt', u'funded_amnt_inv', u'term', u'int_rate',
       u'installment', u'grade', u'emp_length', u'annual_inc', u'dti',
       u'delinq_2yrs', u'inq_last_6mths', u'mths_since_last_delinq',
       u'mths_since_last_record', u'open_acc', u'pub_rec', u'revol_bal',
       u'revol_util', u'total_acc', u'out_prncp', u'out_prncp_inv',
       u'total_pymnt', u'total_pymnt_inv', u'total_rec_prncp',
       u'total_rec_int', u'total_rec_late_fee', u'recoveries',
       u'collection_recovery_fee', u'last_pymnt_amnt',
       u'collections_12_mths_ex_med', u'mths_since_last_major_derog',
       u'annual_inc_joint', u'dti_joint', u'acc_now_delinq', u'tot_coll_amt',
       u'tot_cur_bal', u'open_acc_6m', u'open_il_6m', u'open_il_12m',
       u'open_il_24m', u'mths_since_rcnt_il', u'total_bal_il', u'il_util',
       u'open_rv_12m', u'open_rv_24m', u'max_bal_bc', u'all_util',
       u'total_rev_hi_lim', u'inq_fi', u'total_cu_tl', u'inq_last_12m',
       u'is_Verified', u'is

In [149]:
X = df_strat[df_strat.columns.values]
X.drop('FullyPaid', axis = 1, inplace = True)

y = df_strat.FullyPaid

In [150]:
train_X, test_X, train_y, test_y = cross_validation.train_test_split(X, y, test_size = 0.4)

In [156]:
model = ensemble.RandomForestClassifier(n_estimators = 1000, oob_score = True)

In [157]:
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [ ]:
#areaunderthecurve
#feature importance

In [158]:
train_y.value_counts()

0.0    124663
1.0    124604
Name: FullyPaid, dtype: int64

In [176]:
#cross_validation.cross_val_score(model, train_X, train_y, cv = 10).mean()
#train_X.drop('FullyPaid', axis = 1, inplace = True)



/Users/gfnu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [177]:
model.fit(train_X, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=True, random_state=None, verbose=0, warm_start=False)

In [178]:
model.oob_score_

0.99815860101818532

In [179]:
model.score(train_X, train_y)

1.0

In [180]:
train_X.columns

Index([u'loan_amnt', u'funded_amnt', u'funded_amnt_inv', u'term', u'int_rate',
       u'installment', u'grade', u'emp_length', u'annual_inc', u'dti',
       u'delinq_2yrs', u'inq_last_6mths', u'mths_since_last_delinq',
       u'mths_since_last_record', u'open_acc', u'pub_rec', u'revol_bal',
       u'revol_util', u'total_acc', u'out_prncp', u'out_prncp_inv',
       u'total_pymnt', u'total_pymnt_inv', u'total_rec_prncp',
       u'total_rec_int', u'total_rec_late_fee', u'recoveries',
       u'collection_recovery_fee', u'last_pymnt_amnt',
       u'collections_12_mths_ex_med', u'mths_since_last_major_derog',
       u'annual_inc_joint', u'dti_joint', u'acc_now_delinq', u'tot_coll_amt',
       u'tot_cur_bal', u'open_acc_6m', u'open_il_6m', u'open_il_12m',
       u'open_il_24m', u'mths_since_rcnt_il', u'total_bal_il', u'il_util',
       u'open_rv_12m', u'open_rv_24m', u'max_bal_bc', u'all_util',
       u'total_rev_hi_lim', u'inq_fi', u'total_cu_tl', u'inq_last_12m',
       u'is_Verified', u'is

In [181]:
sorted(zip(model.feature_importances_, train_X.columns.values), reverse = True)

[(0.25411587363224591, 'out_prncp'),
 (0.24320262471959139, 'out_prncp_inv'),
 (0.14501527012508422, 'last_pymnt_amnt'),
 (0.097817865467722878, 'total_rec_prncp'),
 (0.058540116359250022, 'total_pymnt'),
 (0.052908515274339381, 'total_pymnt_inv'),
 (0.017627710043953023, 'tot_cur_bal'),
 (0.017024108732162507, 'funded_amnt_inv'),
 (0.016813148684687539, 'total_rev_hi_lim'),
 (0.015743300682572837, 'recoveries'),
 (0.015156750031103183, 'funded_amnt'),
 (0.013081212198627259, 'loan_amnt'),
 (0.012843317039017924, 'collection_recovery_fee'),
 (0.011417417766420886, 'installment'),
 (0.0066332775806746428, 'total_rec_int'),
 (0.0059265646302222386, 'is_Whole'),
 (0.003048821785753691, 'term'),
 (0.0014217993841495673, 'int_rate'),
 (0.0013663833106657224, 'inq_last_6mths'),
 (0.0012647464633596161, 'dti'),
 (0.0011822504015090451, 'revol_bal'),
 (0.0010023669042630566, 'grade'),
 (0.00065891301706137002, 'revol_util'),
 (0.00062558594262079696, 'annual_inc'),
 (0.00050340882528193795, 't

In [2]:
# out_prncp	Remaining outstanding principal for total amount funded
# out_prncp_inv	Remaining outstanding principal for portion of total amount funded by investors

In [182]:
train_y_hat = model.predict(train_X)

In [183]:
(train_y_hat == train_y).mean()

1.0

In [185]:
#test_X.drop('FullyPaid', axis = 1, inplace = True)

/Users/gfnu/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [186]:
model.score(test_X, test_y)

0.99810445363132527

In [187]:
test_y_hat = model.predict(test_X)

In [188]:
(test_y_hat == test_y).mean()

0.99810445363132527